In [1]:
import numpy as np
import pandas as pd
import novosparc
import matplotlib
import matplotlib.pyplot as plt
from scipy.spatial.distance import cdist
%matplotlib inline

In [2]:
data_path = 'raw/expr_matrix_10day_log_normalized.txt'   # 11566 cells and 2000 genes
output_folder = 'out'
tissue_path = 'raw/myshape.npy'
hvg_path = 'raw/high_variable_genes_10day.txt'

location_marker = 'raw/dge_full.txt'

In [ ]:
# load dataset:  expression matrix   col-gene,row-cell
dataset = novosparc.io.load_data(data_path)

In [ ]:
dataset.shape

In [ ]:
# just to view gene names and the matrix
gene_names = np.asarray(dataset.var.index.tolist())
dge = dataset.X

In [ ]:
dge.shape

In [ ]:
# save the set to dataset.raw
dataset.raw = dataset
# select only genes in hvg_list
dataset, hvg = novosparc.pp.subset_to_hvg(dataset, hvg_file = hvg_path) # dataset[:,dataset.var.index.isin(gene_list)], gene_list

In [ ]:
dataset.shape

In [ ]:
# load target space
# locations = novosparc.geometry.create_target_space_from_image(tissue_path) #

locations = np.load(tissue_path)
locations.shape

In [ ]:
# just to check if atlas is correct
atlas_path = 'raw/atlas.txt'
insitu_genes = np.genfromtxt(atlas_path, usecols=range(29), dtype='str', max_rows=1)
plt.figure(figsize=(20,4))
for i in range(9):
    myimg = np.load('chk/{}.npy'.format(insitu_genes[i].split('.')[0]))
    myloc = locations[myimg==1]

    plt.subplot(1,9,i+1)
    plt.scatter(myloc[:, 0], myloc[:, 1])
    plt.title(insitu_genes[i])
    plt.axis('off')
plt.show()    

In [ ]:
# subsample the locations to save effort -- low resolution now
seed = np.random.choice(locations.shape[0], 3000)  ### -------- later to select markers   3000  11566
locations = locations[seed, :]
locations.shape

In [ ]:
plt.figure(figsize=(6,2))
plt.scatter(locations[:, 1], locations[:, 0])
plt.show()

### Reconstruction
https://github.com/rajewsky-lab/novosparc/blob/master/reconstruct_drosophila_embryo_tutorial.ipynb

In [ ]:
#Reconstruct the tissue de novo/with marker
tissue = novosparc.cm.Tissue(dataset=dataset, locations=locations, output_folder=output_folder)

In [ ]:
atlas_path = 'raw/atlas.txt'
insitu_genes = np.genfromtxt(atlas_path, usecols=range(29), dtype='str', max_rows=1)

In [ ]:
# find indices of marker genes in dge dataset
gene_names = tissue.gene_names
markers_in_sc = np.array([], dtype='int')
markers_in_sc_name = []
for marker in insitu_genes:
    marker_index = np.where(gene_names == marker)[0]
    if len(marker_index) > 0:
        markers_in_sc = np.append(markers_in_sc, marker_index[0])
        markers_in_sc_name.append(marker)

In [ ]:
# load atlas matrix
# make sure atlas_matrix.shape[1]==tissue.dge[:, markers_in_sc].shape[1] (or dataset.dge)
# also don't forget to extract points that used in locations only  (seed)
atlas_matrix = pd.read_csv(atlas_path,sep='\t')
atlas_matrix = atlas_matrix[markers_in_sc_name]
atlas_matrix = atlas_matrix.to_numpy()
atlas_matrix = atlas_matrix[seed,:]
atlas_matrix.shape

In [ ]:
tissue.setup_reconstruction(markers_to_use=markers_in_sc, insitu_matrix=atlas_matrix, num_neighbors_s=5, num_neighbors_t=5)

In [ ]:
tissue.reconstruct(alpha_linear=0.5, epsilon=5e-3) ########调试alpha_linear，似乎是marker gene 的重要程度[0-1]，0是不加marker

In [ ]:
tissue.calculate_spatially_informative_genes()

In [ ]:
sdge_hvg = tissue.sdge

In [ ]:
sdge = tissue.calculate_sdge_for_all_genes()

In [ ]:
# plot some genes
gene_list_to_plot = [
    'SMED30011970', #eye and head, dd_4427
    'SMED30030642', #pharynx
    'SMED30001882',#brain and phx
    'SMED30005457', #super strong; big cells around the gut
    'SMED30000013', #gut
    'SMED30010123', #protonephridia
    'SMED30016244', #secretory cells?
    'SMED30011490' #epithelium
]

In [ ]:
gene_dict = {
    'SMED30011970':'eye and head',
    'SMED30030642':'pharynx',
    'SMED30001882':'brain and phx',
    'SMED30005457':'big cells-gut(++)',
    'SMED30000013':'gut',
    'SMED30010123':'protonephridia',
    'SMED30016244':'secretory cells',
    'SMED30011490':'epithelium'
}

In [ ]:
mylen=len(gene_list_to_plot)
plt.figure(figsize=(2*mylen,6))
for i in range(len(gene_list_to_plot)):
        plt.subplot(1,mylen,i+1)
        plt.scatter(locations[:, 0], locations[:, 1], c=sdge[np.argwhere(gene_names == gene_list_to_plot[i])[0], :].flatten(), s=1)
        plt.title(gene_dict[gene_list_to_plot[i]])
        plt.axis('off')
plt.show()